In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "svm"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
import pandas as pd

In [3]:
## Question 1
## 데이터를 로드하라. 데이터는 \t을 기준으로 구분되어있다.

DataUrl = 'https://raw.githubusercontent.com/Datamanim/pandas/main/lol.csv'
df = pd.read_csv(DataUrl,sep='\t')
type(df)

pandas.core.frame.DataFrame

In [4]:
## Question 2
## 데이터의 상위 5개 행을 출력하라
Ans = df.head(5)
Ans

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18


In [5]:
## Question 3
## 데이터의 행과 열의 갯수를 파악하라

print(df.shape)
print('행:',df.shape[0])
print('열:',df.shape[1])

(51490, 61)
행: 51490
열: 61


In [7]:
df

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51485,3308904636,1503076540231,1944,9,2,1,2,2,0,2,...,10,2,0,4,0,55,-1,90,238,157
51486,3215685759,1496957179355,3304,9,2,1,1,2,2,2,...,11,7,4,4,1,157,55,119,154,105
51487,3322765040,1504029863961,2156,9,2,2,2,2,0,1,...,10,2,0,2,0,113,122,53,11,157
51488,3256675373,1499562036246,1475,9,2,2,2,2,0,2,...,11,3,0,1,0,154,39,51,90,114


In [9]:
## Question 4
## 전체 컬럼을 출력하라
DC = df.columns
DC

Index(['gameId', 'creationTime', 'gameDuration', 'seasonId', 'winner',
       'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron',
       'firstDragon', 'firstRiftHerald', 't1_champ1id', 't1_champ1_sum1',
       't1_champ1_sum2', 't1_champ2id', 't1_champ2_sum1', 't1_champ2_sum2',
       't1_champ3id', 't1_champ3_sum1', 't1_champ3_sum2', 't1_champ4id',
       't1_champ4_sum1', 't1_champ4_sum2', 't1_champ5id', 't1_champ5_sum1',
       't1_champ5_sum2', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills',
       't1_dragonKills', 't1_riftHeraldKills', 't1_ban1', 't1_ban2', 't1_ban3',
       't1_ban4', 't1_ban5', 't2_champ1id', 't2_champ1_sum1', 't2_champ1_sum2',
       't2_champ2id', 't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3id',
       't2_champ3_sum1', 't2_champ3_sum2', 't2_champ4id', 't2_champ4_sum1',
       't2_champ4_sum2', 't2_champ5id', 't2_champ5_sum1', 't2_champ5_sum2',
       't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills',
       't2_riftHer

In [11]:
## Question 5
## 6번째 컬럼명을 출력하라
DC[5]

'firstBlood'

In [15]:
## Question 6
## 6번째 컬럼의 데이터 타입을 확인하라
Ans = df.iloc[:, 5].dtype
Ans

dtype('int64')

In [16]:
## Question 7
## 데이터셋의 인덱스 구성은 어떤가
Ans = df.index
Ans

RangeIndex(start=0, stop=51490, step=1)

In [17]:
## Question 8
## 6번째 컬럼의 3번째 값은 무엇인가?
Ans = df.iloc[2, 5]
Ans

2

In [18]:
## Question 9
## 데이터를 로드하라. 컬럼이 한글이기에 적절한 처리해줘야함
DataUrl = 'https://raw.githubusercontent.com/Datamanim/pandas/main/Jeju.csv'
df = pd.read_csv(DataUrl,encoding='euc-kr')
type(df)

pandas.core.frame.DataFrame

In [20]:
## Question 10
## 데이터 마지막 3개행을 출력하라
Ans = df.tail(3)
Ans

,id,일자,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
9618,32066,2020-04-30,제주시,도두동,28397.481,3144.895,84052.697,115595.073,41.053,29.421,20.3,0.0,3.0
9619,32067,2020-04-30,서귀포시,안덕면,348037.846,29106.286,251129.660,628273.792,46.595,49.189,17.6,0.0,3.5
9620,32068,2020-04-30,제주시,연동,1010643.372,65673.477,447622.068,1523938.917,40.863,27.765,14.1,0.0,4.8


In [21]:
## Question 11
## 수치형 변수를 가진 컬럼을 출력하라
Ans = df.select_dtypes(exclude=object).columns
Ans

Index(['id', '거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도', '평균 소요 시간', '평균 기온',
       '일강수량', '평균 풍속'],
      dtype='object')

In [22]:
## Question 12
## 범주형 변수를 가진 컬럼을 출력하라
Ans = df.select_dtypes(include=object).columns
Ans

Index(['일자', '시도명', '읍면동명'], dtype='object')

In [23]:
## Question 13
## 각 컬럼의 결측치 숫자를 파악하라
Ans = df.isnull().sum()
Ans

id          0
일자          0
시도명         0
읍면동명        0
거주인구        0
근무인구        0
방문인구        0
총 유동인구      0
평균 속도       0
평균 소요 시간    0
평균 기온       0
일강수량        0
평균 풍속       0
dtype: int64

In [24]:
## Question 14
## 각 컬럼의 데이터수, 데이터타입을 한번에 확인하라

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9621 entries, 0 to 9620
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        9621 non-null   int64  
 1   일자        9621 non-null   object 
 2   시도명       9621 non-null   object 
 3   읍면동명      9621 non-null   object 
 4   거주인구      9621 non-null   float64
 5   근무인구      9621 non-null   float64
 6   방문인구      9621 non-null   float64
 7   총 유동인구    9621 non-null   float64
 8   평균 속도     9621 non-null   float64
 9   평균 소요 시간  9621 non-null   float64
 10  평균 기온     9621 non-null   float64
 11  일강수량      9621 non-null   float64
 12  평균 풍속     9621 non-null   float64
dtypes: float64(9), int64(1), object(3)
memory usage: 977.3+ KB


In [25]:
## Question 15
## 각 수치형 변수의 분포(사분위, 평균, 표준편차, 최대 , 최소)를 확인하라
Ans = df.describe()
Ans

,id,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
count,9621.000000,9.621000e+03,9621.000000,9621.000000,9.621000e+03,9621.000000,9621.000000,9621.000000,9621.000000,9621.000000
mean,27258.000000,3.174315e+05,35471.201510,195889.561802,5.487922e+05,41.109084,37.215873,13.550828,6.972426,2.753171
std,2777.487804,2.982079e+05,40381.214775,140706.090325,4.608802e+05,8.758631,12.993786,7.745515,27.617260,1.498538
min,22448.000000,9.305552e+03,1407.936000,11538.322000,2.225181e+04,24.333000,12.667000,-9.600000,0.000000,0.000000
25%,24853.000000,9.539939e+04,12074.498000,99632.153000,2.216910e+05,34.250000,27.889000,7.600000,0.000000,1.700000
50%,27258.000000,2.221105e+05,21960.928000,152805.335000,3.866935e+05,39.640000,34.500000,13.400000,0.000000,2.400000
75%,29663.000000,4.106671e+05,40192.032000,236325.109000,6.406918e+05,49.105000,46.176000,19.700000,1.500000,3.400000
max,32068.000000,1.364504e+06,263476.965000,723459.209000,2.066484e+06,103.000000,172.200000,30.400000,587.500000,13.333000


In [26]:
## Question 16
## 거주인구 컬럼의 값들을 출력하라
Ans = df['거주인구']
Ans

0         32249.987
1        213500.997
2       1212382.218
3         33991.653
4        155036.925
           ...     
9616     228260.005
9617     459959.064
9618      28397.481
9619     348037.846
9620    1010643.372
Name: 거주인구, Length: 9621, dtype: float64

In [27]:
## Question 17
## 평균 속도 컬럼의 4분위 범위(IQR) 값을 구하여라
Ans = df['평균 속도'].quantile(0.75) - df['평균 속도'].quantile(0.25)
Ans

14.854999999999997

In [28]:
## Question 18
## 읍면동명 컬럼의 유일값 갯수를 출력하라
Ans = df.읍면동명.nunique()
Ans

41

In [30]:
## Question 19
## 읍면동명 컬럼의 유일값을 모두 출력하라
Ans = df.읍면동명.unique()
Ans

array(['도두동', '외도동', '이도2동', '일도1동', '대천동', '서홍동', '한경면', '송산동', '조천읍',
       '일도2동', '영천동', '예래동', '대륜동', '삼도1동', '이호동', '건입동', '중앙동', '삼양동',
       '삼도2동', '이도1동', '남원읍', '대정읍', '정방동', '효돈동', '아라동', '한림읍', '구좌읍',
       '용담1동', '오라동', '화북동', '연동', '표선면', '중문동', '성산읍', '안덕면', '천지동',
       '노형동', '동홍동', '용담2동', '봉개동', '애월읍'], dtype=object)

In [31]:
## Question 20
## 데이터를 로드하라.
DataUrl = 'https://raw.githubusercontent.com/Datamanim/pandas/main/chipo.csv'
df = pd.read_csv(DataUrl)
Ans = type(df)
Ans

pandas.core.frame.DataFrame

In [38]:
## Question 21
## quantity컬럼 값이 3인 데이터를 추출하여 첫 5행을 출력하라
Ans = df.loc[df['quantity']==3].head(5)
Ans

,order_id,quantity,item_name,choice_description,item_price
409,178,3,Chicken Bowl,"[[Fresh Tomato Salsa (Mild), Tomatillo-Green C...",$32.94
445,193,3,Bowl,"[Braised Carnitas, Pinto Beans, [Sour Cream, C...",$22.20
689,284,3,Canned Soft Drink,[Diet Coke],$3.75
818,338,3,Bottled Water,NaN,$3.27
850,350,3,Canned Soft Drink,[Sprite],$3.75


In [34]:
## Question 22
## quantity컬럼 값이 3인 데이터를 추출하여 index를 0부터 정렬하고 첫 5행을 출력하라
Ans = df.loc[df['quantity']==3].head(5).reset_index(drop=True)
Ans

In [41]:
## Question 23
## quantity , item_price 두개의 컬럼으로 구성된 새로운 데이터 프레임을 정의하라   
Ans = df[['quantity', 'item_price']]
Ans

,quantity,item_price
0,1,$2.39
1,1,$3.39
2,1,$3.39
3,1,$2.39
4,2,$16.98
...,...,...
4617,1,$11.75
4618,1,$11.75
4619,1,$11.25
4620,1,$8.75


In [42]:
## Question 24
## item_price 컬럼의 달러표시 문자를 제거하고 float 타입으로 저장하여 new_price 컬럼에 저장하라
df['new_price'] = df['item_price'].str[1:].astype('float')
Ans = df['new_price'].head()
Ans

0     2.39
1     3.39
2     3.39
3     2.39
4    16.98
Name: new_price, dtype: float64

In [45]:
## Question 25
## new_price 컬럼이 5이하의 값을 가지는 데이터프레임을 추출하고, 전체 갯수를 구하여라
Ans = len(df.loc[df['new_price']<=5])
Ans

1652

In [51]:
## Question 26
## item_name명이 Chicken Salad Bowl 인 데이터 프레임을 추출하라고 index 값을 초기화 하여라
Ans = df.loc[df['item_name'] == 'Chicken Salad Bowl'].reset_index(drop=True)

In [52]:
Ans

,order_id,quantity,item_name,choice_description,item_price,new_price
0,20,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$8.75,8.75
1,60,2,Chicken Salad Bowl,"[Tomatillo Green Chili Salsa, [Sour Cream, Che...",$22.50,22.50
2,94,2,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$22.50,22.50
3,111,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",$8.75,8.75
4,137,2,Chicken Salad Bowl,"[Fresh Tomato Salsa, Fajita Vegetables]",$17.50,17.50
...,...,...,...,...,...,...
105,1813,2,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$17.50,17.50
106,1822,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Black Beans, Cheese, Gua...",$11.25,11.25
107,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25,11.25
108,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75,8.75


In [58]:
## Question 27
## new_price값이 9 이하이고 item_name 값이 Chicken Salad Bowl 인 데이터 프레임을 추출하라
Ans = df.loc[(df['new_price']<=9) & (df['item_name']=='Chicken Salad Bowl')]
Ans.head()

,order_id,quantity,item_name,choice_description,item_price,new_price
44,20,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$8.75,8.75
256,111,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",$8.75,8.75
526,220,1,Chicken Salad Bowl,"[Roasted Chili Corn Salsa, [Black Beans, Sour ...",$8.75,8.75
528,221,1,Chicken Salad Bowl,"[Tomatillo Green Chili Salsa, [Fajita Vegetabl...",$8.75,8.75
529,221,1,Chicken Salad Bowl,"[Tomatillo Green Chili Salsa, [Fajita Vegetabl...",$8.75,8.75


In [64]:
## Question 28
## df의 new_price 컬럼 값에 따라 오름차순으로 정리하고 index를 초기화 하여라
Ans = df.sort_values('new_price').reset_index(drop=True)
Ans

,order_id,quantity,item_name,choice_description,item_price,new_price
0,471,1,Bottled Water,NaN,$1.09,1.09
1,338,1,Canned Soda,[Coca Cola],$1.09,1.09
2,1575,1,Canned Soda,[Dr. Pepper],$1.09,1.09
3,47,1,Canned Soda,[Dr. Pepper],$1.09,1.09
4,1014,1,Canned Soda,[Coca Cola],$1.09,1.09
...,...,...,...,...,...,...
4617,1443,3,Veggie Burrito,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",$33.75,33.75
4618,1443,4,Chicken Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Chees...",$35.00,35.00
4619,511,4,Chicken Burrito,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",$35.00,35.00
4620,1398,3,Carnitas Bowl,"[Roasted Chili Corn Salsa, [Fajita Vegetables,...",$35.25,35.25


In [69]:
## Question 29
## df의 item_name 컬럼 값중 Chips 포함하는 경우의 데이터를 출력하라
Ans = df.loc[df.item_name.str.contains('Chips')]
Ans.head()

,order_id,quantity,item_name,choice_description,item_price,new_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,2.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,2.39
6,3,1,Side of Chips,NaN,$1.69,1.69
10,5,1,Chips and Guacamole,NaN,$4.45,4.45
14,7,1,Chips and Guacamole,NaN,$4.45,4.45


In [74]:
## Question 30
## df의 짝수번째 컬럼만을 포함하는 데이터프레임을 출력하라
Ans = df.iloc[:,::2]
Ans

,order_id,item_name,item_price
0,1,Chips and Fresh Tomato Salsa,$2.39
1,1,Izze,$3.39
2,1,Nantucket Nectar,$3.39
3,1,Chips and Tomatillo-Green Chili Salsa,$2.39
4,2,Chicken Bowl,$16.98
...,...,...,...
4617,1833,Steak Burrito,$11.75
4618,1833,Steak Burrito,$11.75
4619,1834,Chicken Salad Bowl,$11.25
4620,1834,Chicken Salad Bowl,$8.75


In [77]:
## Question 31
## df의 new_price 컬럼 값에 따라 내림차순으로 정리하고 index를 초기화 하여라
Ans = df.sort_values('new_price',  ascending=False).reset_index(drop=True)
Ans

,order_id,quantity,item_name,choice_description,item_price,new_price
0,1443,15,Chips and Fresh Tomato Salsa,NaN,$44.25,44.25
1,1398,3,Carnitas Bowl,"[Roasted Chili Corn Salsa, [Fajita Vegetables,...",$35.25,35.25
2,511,4,Chicken Burrito,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",$35.00,35.00
3,1443,4,Chicken Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Chees...",$35.00,35.00
4,1443,3,Veggie Burrito,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",$33.75,33.75
...,...,...,...,...,...,...
4617,1578,1,Canned Soda,[Diet Dr. Pepper],$1.09,1.09
4618,1162,1,Bottled Water,NaN,$1.09,1.09
4619,567,1,Canned Soda,[Coca Cola],$1.09,1.09
4620,1014,1,Canned Soda,[Coca Cola],$1.09,1.09


In [81]:
## Question 32
## df의 item_name 컬럼 값이 Steak Salad 또는 Bowl 인 데이터를 인덱싱하라
Ans = df.loc[(df.item_name == 'Steak Salad') | (df.item_name == 'Bowl')]
Ans

,order_id,quantity,item_name,choice_description,item_price,new_price
445,193,3,Bowl,"[Braised Carnitas, Pinto Beans, [Sour Cream, C...",$22.20,22.20
664,276,1,Steak Salad,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$8.99,8.99
673,279,1,Bowl,"[Adobo-Marinated and Grilled Steak, [Sour Crea...",$7.40,7.40
752,311,1,Steak Salad,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$8.99,8.99
893,369,1,Steak Salad,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$8.99,8.99
3502,1406,1,Steak Salad,"[[Lettuce, Fajita Veggies]]",$8.69,8.69


In [82]:
## Question 33
## df의 item_name 컬럼 값이 Steak Salad 또는 Bowl 인 데이터를 데이터 프레임화 한 후, 
## item_name를 기준으로 중복행이 있으면 제거하되 첫번째 케이스만 남겨라
Ans = df.loc[(df.item_name == 'Steak Salad') | (df.item_name == 'Bowl')]
Ans = Ans.drop_duplicates('item_name')
Ans

,order_id,quantity,item_name,choice_description,item_price,new_price
445,193,3,Bowl,"[Braised Carnitas, Pinto Beans, [Sour Cream, C...",$22.20,22.20
664,276,1,Steak Salad,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$8.99,8.99


In [83]:
## Question 34
## df의 item_name 컬럼 값이 Steak Salad 또는 Bowl 인 데이터를 데이터 프레임화 한 후, 
## item_name를 기준으로 중복행이 있으면 제거하되 마지막 케이스만 남겨라
Ans = df.loc[(df.item_name =='Steak Salad') | (df.item_name =='Bowl')]
Ans = Ans.drop_duplicates('item_name', keep='last')
Ans

,order_id,quantity,item_name,choice_description,item_price,new_price
673,279,1,Bowl,"[Adobo-Marinated and Grilled Steak, [Sour Crea...",$7.40,7.40
3502,1406,1,Steak Salad,"[[Lettuce, Fajita Veggies]]",$8.69,8.69


In [87]:
## Question 35
## df의 데이터 중 new_price값이 new_price값의 평균값 이상을 가지는 데이터들을 인덱싱하라
Ans = df.loc[df.new_price >= df.new_price.mean()]
Ans

,order_id,quantity,item_name,choice_description,item_price,new_price
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98,10.98
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75,11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25,9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25,9.25
...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75,11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75,11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25,11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75,8.75


In [89]:
## Question 36
## df의 데이터 중 item_name의 값이 Izze 데이터를 Fizzy Lizzy로 수정하라
df.loc[df.item_name == 'Izze', 'item_name'] = 'Fizzy Lizzy'
Ans = df 
Ans.head()

,order_id,quantity,item_name,choice_description,item_price,new_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,2.39
1,1,1,Fizzy Lizzy,[Clementine],$3.39,3.39
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98


In [100]:
## Question 37
## df의 데이터 중 choice_description 값이 NaN 인 데이터의 갯수를 구하여라
Ans = df.choice_description.isnull().sum()
Ans

Ans = (df.choice_description.isnull()==True).sum()
Ans

1246

In [104]:
## Question 38
## df의 데이터 중 choice_description 값이 NaN 인 데이터를 NoData 값으로 대체하라(loc 이용)

df.loc[df.choice_description.isnull(), 'choice_description'] = 'NoData'
Ans = df
Ans 

,order_id,quantity,item_name,choice_description,item_price,new_price
0,1,1,Chips and Fresh Tomato Salsa,NoData,$2.39,2.39
1,1,1,Fizzy Lizzy,[Clementine],$3.39,3.39
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NoData,$2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98
...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75,11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75,11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25,11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75,8.75


In [111]:
## Question 39
## df의 데이터 중 choice_description 값에 Black이 들어가는 경우를 인덱싱하라
Ans = df[df.choice_description.str.contains('Black')]
Ans.head()

,order_id,quantity,item_name,choice_description,item_price,new_price
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75,11.75
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25,9.25
11,6,1,Chicken Crispy Tacos,"[Roasted Chili Corn Salsa, [Fajita Vegetables,...",$8.75,8.75
12,6,1,Chicken Soft Tacos,"[Roasted Chili Corn Salsa, [Rice, Black Beans,...",$8.75,8.75


In [112]:
Ans = df.loc[df.choice_description.str.contains('Black')]
Ans.head()

,order_id,quantity,item_name,choice_description,item_price,new_price
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75,11.75
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25,9.25
11,6,1,Chicken Crispy Tacos,"[Roasted Chili Corn Salsa, [Fajita Vegetables,...",$8.75,8.75
12,6,1,Chicken Soft Tacos,"[Roasted Chili Corn Salsa, [Rice, Black Beans,...",$8.75,8.75


In [116]:
## Question 40
## df의 데이터 중 choice_description 값에 Vegetables 들어가지 않는 경우의 갯수를 출력하라
Ans = len(df.loc[~df.choice_description.str.contains('Vegetables')])
Ans

3900

In [119]:
## Question 41
## df의 데이터 중 item_name 값이 N으로 시작하는 데이터를 모두 추출하라
Ans = df.loc[df.item_name.str.startswith('N')]
Ans

,order_id,quantity,item_name,choice_description,item_price,new_price
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39
22,11,1,Nantucket Nectar,[Pomegranate Cherry],$3.39,3.39
105,46,1,Nantucket Nectar,[Pineapple Orange Banana],$3.39,3.39
173,77,1,Nantucket Nectar,[Apple],$3.39,3.39
205,91,1,Nantucket Nectar,[Peach Orange],$3.39,3.39
436,189,1,Nantucket Nectar,[Pomegranate Cherry],$3.39,3.39
601,247,2,Nantucket Nectar,[Pineapple Orange Banana],$6.78,6.78
925,381,1,Nantucket Nectar,[Pomegranate Cherry],$3.39,3.39
1356,553,1,Nantucket Nectar,[Pomegranate Cherry],$3.39,3.39
1585,641,1,Nantucket Nectar,[Peach Orange],$3.39,3.39


In [122]:
## Question 42
## df의 데이터 중 item_name 값의 단어갯수가 15개 이상인 데이터를 인덱싱하라
Ans = df[df.item_name.str.len() >= 15]
Ans

,order_id,quantity,item_name,choice_description,item_price,new_price
0,1,1,Chips and Fresh Tomato Salsa,NoData,$2.39,2.39
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NoData,$2.39,2.39
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25,9.25
10,5,1,Chips and Guacamole,NoData,$4.45,4.45
...,...,...,...,...,...,...
4615,1832,1,Chicken Soft Tacos,"[Fresh Tomato Salsa, [Rice, Cheese, Sour Cream]]",$8.75,8.75
4616,1832,1,Chips and Guacamole,NoData,$4.45,4.45
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25,11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75,8.75


In [125]:
## Question 43
## df의 데이터 중 new_price값이 lst에 해당하는 경우의 
## 데이터 프레임을 구하고 그 갯수를 출력하라 lst =[1.69, 2.39, 3.39, 4.45, 9.25, 10.98, 11.75, 16.98]
lst =[1.69, 2.39, 3.39, 4.45, 9.25, 10.98, 11.75, 16.98]
Ans = df.loc[df.new_price.isin(lst)]
display(Ans.head())
print(len(Ans))

,order_id,quantity,item_name,choice_description,item_price,new_price
0,1,1,Chips and Fresh Tomato Salsa,NoData,$2.39,2.39
1,1,1,Fizzy Lizzy,[Clementine],$3.39,3.39
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NoData,$2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98


1393


In [126]:
##############################################Grouping#########################################################
## Question 44
## 데이터를 로드하고 상위 5개 컬럼을 출력하라
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/pandas/main/AB_NYC_2019.csv')
Ans =df.head(5)
Ans

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [128]:
## Question 45
## 데이터의 각 host_name의 빈도수를 구하고 host_name으로 정렬하여 상위 5개를 출력하라
Ans = df.host_name.value_counts().sort_index()
Ans.head()

'Cil                        1
(Ari) HENRY LEE             1
(Email hidden by Airbnb)    6
(Mary) Haiy                 1
-TheQueensCornerLot         1
Name: host_name, dtype: int64

In [132]:
## Question 46
## 데이터의 각 host_name의 빈도수를 구하고 빈도수 기준 내림차순 정렬한 데이터 프레임을 만들어라. 빈도수 컬럼은 counts로 명명하라
Ans = df.groupby('host_name').size().to_frame().rename(columns={0:'counts'}).sort_values('counts', ascending=False)
Ans

,counts
host_name,
Michael,417
David,403
Sonder (NYC),327
John,294
Alex,279
...,...
Jerbean,1
Jerald,1
Jeonghoon,1


In [135]:
## Question 47
## neighbourhood_group의 값에 따른 neighbourhood컬럼 값의 갯수를 구하여라
Ans = df.groupby(['neighbourhood_group', 'neighbourhood'], as_index=False).size()
Ans.head(4)

neighbourhood_group  neighbourhood
Bronx                Allerton         42
                     Baychester        7
                     Belmont          24
                     Bronxdale        19
dtype: int64

In [139]:
## Question 48
## neighbourhood_group의 값에 따른 neighbourhood컬럼 값 중 neighbourhood_group그룹의 최댓값들을 출력하라
Ans = df.groupby(['neighbourhood_group', 'neighbourhood'], as_index=False).size().groupby(['neighbourhood_group'], as_index=False).max()
Ans

TypeError: as_index=False only valid with DataFrame

In [142]:
## Question 49
## neighbourhood_group 값에 따른 price값의 평균, 분산, 최대, 최소 값을 구하여라
Ans = df[['neighbourhood_group','price']].groupby('neighbourhood_group').agg(['mean','var','max','min'])
Ans

price                         
                           mean           var    max min
neighbourhood_group                                     
Bronx                 87.496792  11386.885081   2500   0
Brooklyn             124.383207  34921.719135  10000   0
Manhattan            196.875814  84904.159185  10000   0
Queens                99.517649  27923.130227  10000  10
Staten Island        114.812332  77073.088342   5000  13

In [145]:
## Question 50
## neighbourhood_group 값에 따른 reviews_per_month 평균, 분산, 최대, 최소 값을 구하여라
Ans = df[['neighbourhood_group', 'reviews_per_month']].groupby('neighbourhood_group').agg(['mean', 'var', 'max', 'min'])
Ans

reviews_per_month                       
                                 mean       var    max   min
neighbourhood_group                                         
Bronx                        1.837831  2.799878  10.34  0.02
Brooklyn                     1.283212  2.299040  14.00  0.01
Manhattan                    1.272131  2.651206  58.50  0.01
Queens                       1.941200  4.897848  20.94  0.01
Staten Island                1.872580  2.840895  10.12  0.02

In [147]:
## Question 51
## neighbourhood 값과 neighbourhood_group 값에 따른 price 의 평균을 구하라
Ans = df.groupby(['neighbourhood', 'neighbourhood_group']).price.mean()
Ans

neighbourhood    neighbourhood_group
Allerton         Bronx                   87.595238
Arden Heights    Staten Island           67.250000
Arrochar         Staten Island          115.000000
Arverne          Queens                 171.779221
Astoria          Queens                 117.187778
                                           ...    
Windsor Terrace  Brooklyn               138.993631
Woodhaven        Queens                  67.170455
Woodlawn         Bronx                   60.090909
Woodrow          Staten Island          700.000000
Woodside         Queens                  85.097872
Name: price, Length: 221, dtype: float64

In [148]:
## Question 52
## neighbourhood 값과 neighbourhood_group 값에 따른 price 의 평균을 계층적 indexing 없이 구하라
Ans = df.groupby(['neighbourhood', 'neighbourhood_group']).price.mean().unstack()
Ans

neighbourhood_group,Bronx,Brooklyn,Manhattan,Queens,Staten Island
neighbourhood,,,,,
Allerton,87.595238,NaN,NaN,NaN,NaN
Arden Heights,NaN,NaN,NaN,NaN,67.25
Arrochar,NaN,NaN,NaN,NaN,115.00
Arverne,NaN,NaN,NaN,171.779221,NaN
Astoria,NaN,NaN,NaN,117.187778,NaN
...,...,...,...,...,...
Windsor Terrace,NaN,138.993631,NaN,NaN,NaN
Woodhaven,NaN,NaN,NaN,67.170455,NaN
Woodlawn,60.090909,NaN,NaN,NaN,NaN


In [149]:
## Question 53
## neighbourhood 값과 neighbourhood_group 값에 따른 price 의 평균을 계층적 indexing 없이 구하고 nan 값은 -999값으로 채워라
Ans = df.groupby(['neighbourhood','neighbourhood_group']).price.mean().unstack().fillna(-999)
Ans.head()

neighbourhood_group,Bronx,Brooklyn,Manhattan,Queens,Staten Island
neighbourhood,,,,,
Allerton,87.595238,-999.0,-999.0,-999.000000,-999.00
Arden Heights,-999.000000,-999.0,-999.0,-999.000000,67.25
Arrochar,-999.000000,-999.0,-999.0,-999.000000,115.00
Arverne,-999.000000,-999.0,-999.0,171.779221,-999.00
Astoria,-999.000000,-999.0,-999.0,117.187778,-999.00


In [151]:
## Question 54
##데이터중 neighbourhood_group 값이 Queens값을 가지는 데이터들 중 neighbourhood 그룹별로 price값의 평균, 분산, 최대, 최소값을 구하라

Ans = df[df.neighbourhood_group=='Queens'].groupby(['neighbourhood']).price.agg(['mean', 'var', 'max', 'min'])
Ans.head()

,mean,var,max,min
neighbourhood,,,,
Arverne,171.779221,37383.411141,1500,35
Astoria,117.187778,122428.811196,10000,25
Bay Terrace,142.000000,6816.400000,258,32
Bayside,157.948718,166106.470985,2600,30
Bayswater,87.470588,2330.889706,230,45


In [152]:
## Question 55
## 데이터중 neighbourhood_group 값에 따른 room_type 컬럼의 숫자를 구하고 neighbourhood_group 값을 기준으로 각 값의 비율을 구하여라
Ans = df[['neighbourhood_group','room_type']].groupby(['neighbourhood_group','room_type']).size().unstack()
Ans.loc[:,:] = (Ans.values /Ans.sum(axis=1).values.reshape(-1,1))
Ans

room_type,Entire home/apt,Private room,Shared room
neighbourhood_group,,,
Bronx,0.347388,0.597617,0.054995
Brooklyn,0.475478,0.503979,0.020543
Manhattan,0.609344,0.368496,0.022160
Queens,0.369926,0.595129,0.034945
Staten Island,0.471850,0.504021,0.024129


In [4]:
################APPLY, MAP##################################

## Question 56
## 데이터를 로드하고 데이터 행과 열의 갯수를 출력하라

import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/pandas/main/BankChurnersUp.csv',index_col=0)
Ans =df.shape
Ans

(10127, 18)

In [6]:
##Question 57
## Income_Category의 카테고리를 map 함수를 이용하여 다음과 같이 변경하여 newIncome 컬럼에 매핑하라
## Unknown : N Less than 40K:a40K - 60K:b60K - 80K:c80K - 120K:d120K +’ : e
dic = {
    'Unknown'        : 'N',
    'Less than $40K' : 'a',
    '$40K - $60K'    : 'b',
    '$60K - $80K'    : 'c',
    '$80K - $120K'   : 'd',
    '$120K +'        : 'e'   
}

df['newIncome']  =df.Income_Category.map(lambda x: dic[x])

Ans = df['newIncome']
Ans.head()

0    c
1    a
2    d
3    a
4    c
Name: newIncome, dtype: object

In [7]:
## Question 58
## Income_Category의 카테고리를 apply 함수를 이용하여 다음과 같이 변경하여 
## newIncome 컬럼에 매핑하라 Unknown : N Less than 40K:a40K - 60K:b60K - 80K:c80K - 120K:d120K +’ : e
def changeCategory(x):
    if x =='Unknown':
        return 'N'
    elif x =='Less than $40K':
        return 'a'
    elif x =='$40K - $60K':   
        return 'b'
    elif x =='$60K - $80K':    
        return 'c'
    elif x =='$80K - $120K':   
        return 'd'
    elif x =='$120K +' :     
        return 'e'

df['newIncome']  = df.Income_Category.apply(changeCategory)

Ans = df['newIncome']
Ans.head()

0    c
1    a
2    d
3    a
4    c
Name: newIncome, dtype: object

In [10]:
## Question 59
## Customer_Age의 값을 이용하여 나이 구간을 AgeState 컬럼으로 정의하라. 
## (0~9 : 0 , 10~19 :10 , 20~29 :20 … 각 구간의 빈도수를 출력하라
df['AgeState'] = df.Customer_Age.map(lambda x: x//10 *10)
Ans = df['AgeState'].value_counts().sort_index()
Ans

20     195
30    1841
40    4561
50    2998
60     530
70       2
Name: AgeState, dtype: int64

In [12]:
## Question 60
## Education_Level의 값중 Graduate단어가 포함되는 값은 1 그렇지 않은 경우에는 0으로 변경하여 newEduLevel 컬럼을 정의하고 
## 빈도수를 출력하라
df['newEduLevel'] = df.Education_Level.map(lambda x : 1 if 'Graduate' in x else 0)
Ans = df['newEduLevel'].value_counts()
Ans

0    6483
1    3644
Name: newEduLevel, dtype: int64

In [15]:
## Question 61
## Credit_Limit 컬럼값이 4500 이상인 경우 1 그외의 경우에는 모두 0으로 하는 newLimit 정의하라. newLimit 각 값들의 빈도수를 출력하라
df['newLimit'] = df.Credit_Limit.map(lambda x : 1  if x >= 4500 else 0)
Ans = df['newLimit'].value_counts()
Ans.head()

1    5096
0    5031
Name: newLimit, dtype: int64

In [16]:
## Question 62
## Marital_Status 컬럼값이 Married 이고 Card_Category 컬럼의 값이 Platinum인 경우 1 그외의 경우에는 
## 모두 0으로 하는 newState컬럼을 정의하라. newState의 각 값들의 빈도수를 출력하라
def check(x):
    if x.Marital_Status == 'Married' and x.Card_Category == 'Platinum':
        return 1 
    else:
        return 0 
    
df['newState'] = df.apply(check, axis=1)
Ans = df['newState'].value_counts()
Ans

0    10120
1        7
Name: newState, dtype: int64

In [18]:
## Question 63
## Gender 컬럼값 M인 경우 male F인 경우 female로 값을 변경하여 Gender 컬럼에 새롭게 정의하라. 각 value의 빈도를 출력하라
def changeGender(x):
    if x =='M':
        return 'male'
    else: 
        return 'female'

df['Gender'] = df.Gender.apply(changeGender)
Ans = df['Gender'].value_counts()
Ans

female    10127
Name: Gender, dtype: int64

In [19]:
###############Time_Series############################################
## Question 64
## 데이터를 로드하고 각 열의 데이터 타입을 파악하라
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/pandas/main/timeTest.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574 entries, 0 to 6573
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Yr_Mo_Dy  6574 non-null   object 
 1   RPT       6568 non-null   float64
 2   VAL       6571 non-null   float64
 3   ROS       6572 non-null   float64
 4   KIL       6569 non-null   float64
 5   SHA       6572 non-null   float64
 6   BIR       6574 non-null   float64
 7   DUB       6571 non-null   float64
 8   CLA       6572 non-null   float64
 9   MUL       6571 non-null   float64
 10  CLO       6573 non-null   float64
 11  BEL       6574 non-null   float64
 12  MAL       6570 non-null   float64
dtypes: float64(12), object(1)
memory usage: 667.8+ KB


In [20]:
## Question 65
## Yr_Mo_Dy을 판다스에서 인식할 수 있는 datetime64타입으로 변경하라
df.Yr_Mo_Dy = pd.to_datetime(df.Yr_Mo_Dy)
Ans = df.Yr_Mo_Dy
Ans.head()

0   2061-01-01
1   2061-01-02
2   2061-01-03
3   2061-01-04
4   2061-01-05
Name: Yr_Mo_Dy, dtype: datetime64[ns]

In [21]:
## Question 66
## Yr_Mo_Dy에 존재하는 년도의 유일값을 모두 출력하라
Ans = df.Yr_Mo_Dy.dt.year.unique()
Ans

array([2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978], dtype=int64)

In [24]:
## Question 67
## Yr_Mo_Dy에 년도가 2061년 이상의 경우에는 모두 잘못된 데이터이다. 해당경우의 값은 100을 빼서 새롭게 날짜를 Yr_Mo_Dy 컬럼에 정의하라
def fix_century(x):
    import datetime
    
    year = x.year - 100 if x.year > 1989 else x.year
    return pd.to_datetime(datetime.date(year, x.month, x.day))

df['Yr_Mo_Dy'] = df['Yr_Mo_Dy'].apply(fix_century)

Ans = df.head()
Ans

,Yr_Mo_Dy,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
0,1961-01-01,15.04,14.96,13.17,9.29,NaN,9.87,13.67,10.25,10.83,12.58,18.50,15.04
1,1961-01-02,14.71,NaN,10.83,6.50,12.62,7.67,11.50,10.04,9.79,9.67,17.54,13.83
2,1961-01-03,18.50,16.88,12.33,10.13,11.17,6.17,11.25,NaN,8.50,7.67,12.75,12.71
3,1961-01-04,10.58,6.63,11.75,4.58,4.54,2.88,8.63,1.79,5.83,5.88,5.46,10.88
4,1961-01-05,13.33,13.25,11.42,6.17,10.71,8.21,11.92,6.54,10.92,10.34,12.92,11.83


In [28]:
## Question 68
## 년도별 각컬럼의 평균값을 구하여라
Ans = df.groupby(df.Yr_Mo_Dy.dt.year).mean()
Ans

,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL
Yr_Mo_Dy,,,,,,,,,,,,
1961,12.299583,10.351796,11.362369,6.958227,10.881763,7.729726,9.733923,8.858788,8.647652,9.835577,13.502795,13.680773
1962,12.246923,10.110438,11.732712,6.960440,10.657918,7.393068,11.020712,8.793753,8.316822,9.676247,12.930685,14.323956
1963,12.813452,10.836986,12.541151,7.330055,11.724110,8.434712,11.075699,10.336548,8.903589,10.224438,13.638877,14.999014
1964,12.363661,10.920164,12.104372,6.787787,11.454481,7.570874,10.259153,9.467350,7.789016,10.207951,13.740546,14.910301
1965,12.451370,11.075534,11.848767,6.858466,11.024795,7.478110,10.618712,8.879918,7.907425,9.918082,12.964247,15.591644
1966,13.461973,11.557205,12.020630,7.345726,11.805041,7.793671,10.579808,8.835096,8.514438,9.768959,14.265836,16.307260
1967,12.737151,10.990986,11.739397,7.143425,11.630740,7.368164,10.652027,9.325616,8.645014,9.547425,14.774548,17.135945
1968,11.835628,10.468197,11.409754,6.477678,10.760765,6.067322,8.859180,8.255519,7.224945,7.832978,12.808634,15.017486
1969,11.166356,9.723699,10.902000,5.767973,9.873918,6.189973,8.564493,7.711397,7.924521,7.754384,12.621233,15.762904


In [29]:
## Question 69
## weekday컬럼을 만들고 요일별로 매핑하라 ( 월요일: 0 ~ 일요일 :6)
df['weekday'] = df.Yr_Mo_Dy.dt.weekday
Ans = df['weekday'].head().to_frame()
Ans

,weekday
0,6
1,0
2,1
3,2
4,3


In [33]:
## Question 70
## weekday컬럼을 기준으로 주말이면 1 평일이면 0의 값을 가지는 WeekCheck 컬럼을 만들어라

df['WeekCheck'] = df['weekday'].map(lambda x : 1 if x in [5, 6] else 0)

Ans = df['WeekCheck'].head().to_frame()

Ans

,WeekCheck
0,1
1,0
2,0
3,0
4,0


In [35]:
## Question 71
## 년도, 일자 상관없이 모든 컬럼의 각 달의 평균을 구하여라

Ans = df.groupby(df.Yr_Mo_Dy.dt.month).mean()
Ans.head()

,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL,weekday,WeekCheck
Yr_Mo_Dy,,,,,,,,,,,,,,
1,14.847325,12.914560,13.299624,7.199498,11.667734,8.054839,11.819355,9.512047,9.543208,10.053566,14.550520,18.028763,2.989247,0.284946
2,13.710906,12.111122,12.879132,6.942411,11.551772,7.633858,11.206024,9.341437,9.313169,9.518051,13.728898,17.156142,3.005906,0.287402
3,13.158687,11.505842,12.648118,7.265907,11.554516,7.959409,11.310179,9.635896,9.700324,10.096953,13.810609,16.909317,3.000000,0.283154
4,12.555648,10.429759,12.204815,6.898037,10.677667,7.441389,10.221315,8.909056,8.930870,9.158019,12.664759,14.937611,3.011111,0.288889
5,11.724032,10.145619,11.550394,6.307487,10.224301,6.942061,8.797738,8.452903,8.040806,8.524857,12.767258,13.736039,2.982079,0.283154


In [36]:
## Question 72
## 모든 결측치는 컬럼기준 직전의 값으로 대체하고 첫번째 행에 결측치가 있을경우 뒤에있는 값으로 대채하라
df = df.fillna(method='ffill').fillna(method='bfill')
df.isnull().sum()

Yr_Mo_Dy     0
RPT          0
VAL          0
ROS          0
KIL          0
SHA          0
BIR          0
DUB          0
CLA          0
MUL          0
CLO          0
BEL          0
MAL          0
weekday      0
WeekCheck    0
dtype: int64

In [38]:
## Question 73
## 년도 - 월을 기준으로 모든 컬럼의 평균값을 구하여라
Ans = df.groupby(df.Yr_Mo_Dy.dt.to_period('M')).mean()
Ans.head()

,RPT,VAL,ROS,KIL,SHA,BIR,DUB,CLA,MUL,CLO,BEL,MAL,weekday,WeekCheck
Yr_Mo_Dy,,,,,,,,,,,,,,
1961-01,14.932581,12.084194,13.431613,7.736774,11.215806,8.588065,11.184839,9.270968,9.085806,10.107419,13.880968,14.703226,2.935484,0.290323
1961-02,16.269286,14.975357,14.324643,9.116786,13.852143,10.937500,11.890714,11.846071,11.821429,12.714286,18.583214,15.411786,3.000000,0.285714
1961-03,11.015806,11.296452,10.752903,7.137742,10.509355,8.866774,9.644194,9.829677,10.646129,11.251935,16.410968,15.637742,3.000000,0.258065
1961-04,10.722667,9.427667,9.998000,5.830667,8.435000,6.495000,6.925333,7.094667,7.342333,7.237000,11.147333,10.278333,3.166667,0.333333
1961-05,9.860968,8.735161,10.818065,5.892258,9.490323,6.574839,7.477097,8.177097,8.039355,8.499355,11.900323,12.011613,2.806452,0.258065


In [39]:
## Question 74
## RPT 컬럼의 값을 일자별 기준으로 1차차분하라
Ans = df['RPT'].diff()
Ans.head(3)

0     NaN
1   -0.33
2    3.79
Name: RPT, dtype: float64

In [40]:
## Question 75
## RPT와 VAL의 컬럼을 일주일 간격으로 각각 이동평균한값을 구하여라
Ans= df[['RPT','VAL']].rolling(7).mean()
Ans.head(9)

,RPT,VAL
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,14.124286,12.727143
7,13.541429,11.982857
8,13.237143,11.392857


In [41]:
## Question 76
## 년-월-일:시 컬럼을 pandas에서 인식할 수 있는 datetime 형태로 변경하라. 서울시의 제공데이터의 경우 0시가 24시로 표현된다
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/pandas/main/seoul_pm.csv')
def change_date(x):
    import datetime
    hour = x.split(':')[1]
    date = x.split(":")[0]
    
    if hour =='24':
        hour ='00:00:00'
        
        FinalDate = pd.to_datetime(date +" "+hour) +datetime.timedelta(days=1)
        
    else:
        hour = hour +':00:00'
        FinalDate = pd.to_datetime(date +" "+hour)
    
    return FinalDate

df['(년-월-일:시)'] = df['(년-월-일:시)'].apply(change_date)

Ans = df
Ans.head(3)

,(년-월-일:시),PM10등급,PM10,PM2.5등급,PM2.5,오존등급,오존,이산화질소등급,이산화질소,일산화탄소등급,일산화탄소,아황산가스등급,아황산가스
0,2021-05-15 15:00:00,보통,47.0,보통,19.0,좋음,0.017,좋음,0.023,좋음,0.4,좋음,0.003
1,2021-05-15 14:00:00,보통,43.0,보통,20.0,좋음,0.024,좋음,0.019,좋음,0.3,좋음,0.003
2,2021-05-15 13:00:00,보통,34.0,보통,24.0,보통,0.035,좋음,0.017,좋음,0.4,좋음,0.004


In [45]:
## Question 77
## 일자별 영어요일 이름을 dayName 컬럼에 저장하라
df['dayName'] = df['(년-월-일:시)'].dt.day_name()
Ans = df['dayName']
df

,(년-월-일:시),PM10등급,PM10,PM2.5등급,PM2.5,오존등급,오존,이산화질소등급,이산화질소,일산화탄소등급,일산화탄소,아황산가스등급,아황산가스,dayName
0,2021-05-15 15:00:00,보통,47.0,보통,19.0,좋음,0.017,좋음,0.023,좋음,0.4,좋음,0.003,Saturday
1,2021-05-15 14:00:00,보통,43.0,보통,20.0,좋음,0.024,좋음,0.019,좋음,0.3,좋음,0.003,Saturday
2,2021-05-15 13:00:00,보통,34.0,보통,24.0,보통,0.035,좋음,0.017,좋음,0.4,좋음,0.004,Saturday
3,2021-05-15 12:00:00,보통,41.0,보통,27.0,보통,0.037,좋음,0.020,좋음,0.4,좋음,0.004,Saturday
4,2021-05-15 11:00:00,보통,51.0,보통,34.0,보통,0.033,좋음,0.023,좋음,0.4,좋음,0.005,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,2021-03-25 05:00:00,보통,39.0,보통,18.0,좋음,0.026,좋음,0.025,좋음,0.4,좋음,0.003,Thursday
1235,2021-03-25 04:00:00,보통,34.0,좋음,15.0,좋음,0.017,보통,0.033,좋음,0.4,좋음,0.002,Thursday
1236,2021-03-25 03:00:00,보통,35.0,좋음,13.0,좋음,0.029,좋음,0.025,좋음,0.4,좋음,0.003,Thursday
1237,2021-03-25 02:00:00,보통,35.0,좋음,13.0,보통,0.031,좋음,0.025,좋음,0.3,좋음,0.003,Thursday


In [52]:
## Question 78
## 일자별 각 PM10등급의 빈도수를 파악하라
Ans1 = df.groupby(['dayName', 'PM10등급'], as_index=False).size()
Ans1.head()

dayName  PM10등급
Friday   나쁨         31
         매우나쁨       17
         보통        120
         좋음         21
Monday   나쁨          1
dtype: int64

In [53]:
## Question 79
## 시간이 연속적으로 존재하며 결측치가 없는지 확인하라
# 시간을 차분했을 경우 첫 값은 nan, 이후 모든 차분값이 동일하면 연속이라 판단한다.
check = len(df['(년-월-일:시)'].diff().unique())
if check ==2:
    Ans =True
else:
    Ans = False
    
Ans

True

In [54]:
## Question 80
## 오전 10시와 오후 10시(22시)의 PM10의 평균값을 각각 구하여라
Ans = df.groupby(df['(년-월-일:시)'].dt.hour).mean().iloc[[10,22],[0]]
Ans

,PM10
(년-월-일:시),
10,70.384615
22,69.941176


In [57]:
## Question 81
## 날짜 컬럼을 index로 만들어라
df.set_index('(년-월-일:시)',inplace=True,drop=True)
Ans=df


KeyError: "None of ['(년-월-일:시)'] are in the columns"

In [58]:
Ans.head()

,PM10등급,PM10,PM2.5등급,PM2.5,오존등급,오존,이산화질소등급,이산화질소,일산화탄소등급,일산화탄소,아황산가스등급,아황산가스,dayName
(년-월-일:시),,,,,,,,,,,,,
2021-05-15 15:00:00,보통,47.0,보통,19.0,좋음,0.017,좋음,0.023,좋음,0.4,좋음,0.003,Saturday
2021-05-15 14:00:00,보통,43.0,보통,20.0,좋음,0.024,좋음,0.019,좋음,0.3,좋음,0.003,Saturday
2021-05-15 13:00:00,보통,34.0,보통,24.0,보통,0.035,좋음,0.017,좋음,0.4,좋음,0.004,Saturday
2021-05-15 12:00:00,보통,41.0,보통,27.0,보통,0.037,좋음,0.020,좋음,0.4,좋음,0.004,Saturday
2021-05-15 11:00:00,보통,51.0,보통,34.0,보통,0.033,좋음,0.023,좋음,0.4,좋음,0.005,Saturday


In [59]:
## Question 82
## 데이터를 주단위로 뽑아서 최소,최대 평균, 표준표차를 구하여라
Ans = df.resample('W').agg(['min', 'max', 'mean', 'std'])
Ans.head()

PM10                               PM2.5                    \
             min    max       mean         std   min    max       mean   
(년-월-일:시)                                                                
2021-03-28   6.0  160.0  72.747368   43.345462   4.0  113.0  44.705263   
2021-04-04   3.0  598.0  97.148810  129.911976   1.0  120.0  23.168675   
2021-04-11  17.0  102.0  41.059524   16.325911   7.0   70.0  21.761905   
2021-04-18   3.0  367.0  48.180723   43.254468   2.0   38.0  17.066265   
2021-04-25  17.0  126.0  55.119048   26.659936   7.0   61.0  26.392857   

                          오존         ...     이산화질소           일산화탄소       \
                  std    min    max  ...      mean       std   min  max   
(년-월-일:시)                            ...                                  
2021-03-28  29.551928  0.002  0.085  ...  0.044579  0.023722   0.3  1.4   
2021-04-04  22.399578  0.003  0.055  ...  0.027929  0.014978   0.3  0.9   
2021-04-11  11.479343  0.009  0.070  ...  0.022583  0.009562   0.3  0.7   
2021-04-18   7.867952  0.002  0.070  ...  0.023753  0.013553   0.3  0.6   
2021-04-25  13.094788  0.006  0.090  ...  0.028571  0.014640   0.3  0.8   

                                아황산가스                             
                mean       std    min    max      mean       std  
(년-월-일:시)                                                         
2021-03-28  0.611579  0.232408  0.002  0.006  0.003274  0.000961  
2021-04-04  0.445833  0.135741  0.002  0.004  0.002732  0.000541  
2021-04-11  0.389286  0.087573  0.002  0.004  0.002744  0.000569  
2021-04-18  0.386747  0.084954  0.002  0.004  0.002464  0.000579  
2021-04-25  0.457143  0.122142  0.001  0.011  0.003631  0.001763  

[5 rows x 24 columns]

In [60]:
####################### PIVOT ###############################################
## Question 83
## Indicator을 삭제하고 First Tooltip 컬럼에서 신뢰구간에 해당하는 표현을 지워라
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/pandas/main/under5MortalityRate.csv')
df.drop('Indicator',axis=1,inplace=True)
df['First Tooltip'] = df['First Tooltip'].map(lambda x: float(x.split("[")[0]))
Ans = df
Ans.head(4)

,Location,Period,Dim1,First Tooltip
0,Afghanistan,2019,Both sexes,60.27
1,Afghanistan,2019,Male,63.83
2,Afghanistan,2019,Female,56.57
3,Afghanistan,2018,Both sexes,62.54


In [62]:
## Question 84
## 년도가 2015년 이상, Dim1이 Both sexes인 케이스만 추출하라
target = df[(df.Period >= 2015) & (df.Dim1 == 'Both sexes')]
Ans=target
Ans.head()

,Location,Period,Dim1,First Tooltip
0,Afghanistan,2019,Both sexes,60.27
3,Afghanistan,2018,Both sexes,62.54
6,Afghanistan,2017,Both sexes,64.94
9,Afghanistan,2016,Both sexes,67.57
12,Afghanistan,2015,Both sexes,70.44


In [63]:
## Question 85
## 84번 문제에서 추출한 데이터로 아래와 같이 나라에 따른 년도별 사망률을 데이터 프레임화 하라
Ans = target.pivot(index='Location',columns='Period',values='First Tooltip')
Ans.head(3)

Period,2015,2016,2017,2018,2019
Location,,,,,
Afghanistan,70.44,67.57,64.94,62.54,60.27
Albania,9.57,9.42,9.42,9.53,9.68
Algeria,25.18,24.79,24.32,23.81,23.26


In [64]:
## Question 86
## Dim1에 따른 년도별 사망비율의 평균을 구하라
Ans = df.pivot_table(index='Dim1',columns='Period',values='First Tooltip',aggfunc='mean')
Ans.iloc[:,:4]

Period,1950,1951,1952,1953
Dim1,,,,
Both sexes,147.700816,155.537544,157.811094,156.147206
Female,140.909796,149.210175,151.516094,150.250882
Male,154.151224,161.538246,163.760781,161.742059


In [65]:
## Question 87
## 데이터에서 한국 KOR 데이터만 추출하라
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/pandas/main/winter.csv')
kr = df[df.Country=='KOR']
Ans = kr
Ans.head(4)

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
2652,1992,Albertville,Skating,Short Track Speed Skating,"LEE, Jun-Ho",KOR,Men,1000M,Bronze
2653,1992,Albertville,Skating,Short Track Speed Skating,"KIM, Ki-Hoon",KOR,Men,1000M,Gold
2671,1992,Albertville,Skating,Short Track Speed Skating,"KIM, Ki-Hoon",KOR,Men,5000M Relay,Gold
2672,1992,Albertville,Skating,Short Track Speed Skating,"LEE, Jun-Ho",KOR,Men,5000M Relay,Gold


In [66]:
## Question 88
## 한국 올림픽 메달리스트 데이터에서 년도에 따른 medal 갯수를 데이터프레임화 하라
Ans = kr.pivot_table(index='Year',columns='Medal',aggfunc='size').fillna(0)
Ans

Medal,Bronze,Gold,Silver
Year,,,
1992,1.0,5.0,1.0
1994,1.0,8.0,1.0
1998,2.0,6.0,4.0
2002,0.0,5.0,2.0
2006,2.0,14.0,3.0
2010,2.0,6.0,10.0
2014,2.0,7.0,5.0


In [71]:
## Question 89
## 전체 데이터에서 sport종류에 따른 성별수를 구하여라
Ans = df.pivot_table(index='Sport',columns='Gender',aggfunc='size')
Ans

Gender,Men,Women
Sport,,
Biathlon,270,150
Bobsleigh,416,36
Curling,97,75
Ice Hockey,1231,305
Luge,135,45
Skating,665,564
Skiing,1130,651


In [69]:
df

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1924,Chamonix,Biathlon,Biathlon,"BERTHET, G.",FRA,Men,Military Patrol,Bronze
1,1924,Chamonix,Biathlon,Biathlon,"MANDRILLON, C.",FRA,Men,Military Patrol,Bronze
2,1924,Chamonix,Biathlon,Biathlon,"MANDRILLON, Maurice",FRA,Men,Military Patrol,Bronze
3,1924,Chamonix,Biathlon,Biathlon,"VANDELLE, André",FRA,Men,Military Patrol,Bronze
4,1924,Chamonix,Biathlon,Biathlon,"AUFDENBLATTEN, Adolf",SUI,Men,Military Patrol,Gold
...,...,...,...,...,...,...,...,...,...
5765,2014,Sochi,Skiing,Snowboard,"JONES, Jenny",GBR,Women,Slopestyle,Bronze
5766,2014,Sochi,Skiing,Snowboard,"ANDERSON, Jamie",USA,Women,Slopestyle,Gold
5767,2014,Sochi,Skiing,Snowboard,"MALTAIS, Dominique",CAN,Women,Snowboard Cross,Silver
5768,2014,Sochi,Skiing,Snowboard,"SAMKOVA, Eva",CZE,Women,Snowboard Cross,Gold


In [72]:
## Question 90
## 전체 데이터에서 Discipline종류에 따른 따른 Medal수를 구하여라
Ans = df.pivot_table(index='Discipline',columns='Medal',aggfunc='size')
Ans

Medal,Bronze,Gold,Silver
Discipline,,,
Alpine Skiing,141,143,144
Biathlon,139,140,141
Bobsleigh,147,134,141
Cross Country Skiing,263,264,262
Curling,56,58,58
Figure skating,118,122,119
Freestyle Skiing,34,34,34
Ice Hockey,512,510,514
Luge,60,62,58


In [73]:
###################MERGE, CONCAT########################
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/pandas/main/mergeTEst.csv',index_col= 0)
df1 = df.iloc[:4,:]
df2 = df.iloc[4:,:]

display(df1)
display(df2)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,,,,
Afghanistan,64.023,61.640,59.367,57.170,55.08,53.107,51.267,49.560,47.983,46.453
Albania,11.803,10.807,9.943,9.267,8.79,8.493,8.363,8.363,8.453,8.597
Algeria,23.540,22.907,22.450,22.117,21.85,21.587,21.257,20.850,20.407,19.930
Andorra,4.240,4.033,3.843,3.667,3.49,3.330,3.187,3.060,2.933,2.827


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,,,,
Angola,75.713,71.280,67.233,63.570,60.430,57.757,55.510,53.460,51.757,50.093
Antigua and Barbuda,8.667,8.223,7.807,7.420,7.070,6.757,6.483,6.230,6.000,5.783
Argentina,12.887,12.380,11.840,11.283,10.733,10.203,9.683,9.177,8.680,8.227
Armenia,16.497,15.677,14.897,14.170,13.477,12.817,12.183,11.583,11.007,10.497
Australia,3.993,3.803,3.623,3.467,3.343,3.253,3.183,3.137,3.090,3.047
Austria,3.573,3.463,3.333,3.210,3.113,3.043,2.987,2.943,2.897,2.843


In [75]:
## Question 91
## df1과 df2 데이터를 하나의 데이터 프레임으로 합쳐라
total = pd.concat([df1,df2])
Ans = total
Ans

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,,,,
Afghanistan,64.023,61.640,59.367,57.170,55.080,53.107,51.267,49.560,47.983,46.453
Albania,11.803,10.807,9.943,9.267,8.790,8.493,8.363,8.363,8.453,8.597
Algeria,23.540,22.907,22.450,22.117,21.850,21.587,21.257,20.850,20.407,19.930
Andorra,4.240,4.033,3.843,3.667,3.490,3.330,3.187,3.060,2.933,2.827
Angola,75.713,71.280,67.233,63.570,60.430,57.757,55.510,53.460,51.757,50.093
Antigua and Barbuda,8.667,8.223,7.807,7.420,7.070,6.757,6.483,6.230,6.000,5.783
Argentina,12.887,12.380,11.840,11.283,10.733,10.203,9.683,9.177,8.680,8.227
Armenia,16.497,15.677,14.897,14.170,13.477,12.817,12.183,11.583,11.007,10.497
Australia,3.993,3.803,3.623,3.467,3.343,3.253,3.183,3.137,3.090,3.047


In [76]:
df3 = df.iloc[:2,:4]
df4 = df.iloc[5:,3:]

display(df3)
display(df4)

,2010,2011,2012,2013
Location,,,,
Afghanistan,64.023,61.640,59.367,57.170
Albania,11.803,10.807,9.943,9.267


,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,
Antigua and Barbuda,7.420,7.070,6.757,6.483,6.230,6.000,5.783
Argentina,11.283,10.733,10.203,9.683,9.177,8.680,8.227
Armenia,14.170,13.477,12.817,12.183,11.583,11.007,10.497
Australia,3.467,3.343,3.253,3.183,3.137,3.090,3.047
Austria,3.210,3.113,3.043,2.987,2.943,2.897,2.843


In [77]:
## Question 92

## df3과 df4 데이터를 하나의 데이터 프레임으로 합쳐라. 둘다 포함하고 있는 년도에 대해서만 고려한다
Ans = pd.concat([df3,df4],join='inner')
Ans

,2013
Location,
Afghanistan,57.170
Albania,9.267
Antigua and Barbuda,7.420
Argentina,11.283
Armenia,14.170
Australia,3.467
Austria,3.210


In [78]:
## Question 93
## df3과 df4 데이터를 하나의 데이터 프레임으로 합쳐라. 모든 컬럼을 포함하고, 결측치는 0으로 대체한다
Ans = pd.concat([df3,df4],join='outer').fillna(0)
Ans

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Location,,,,,,,,,,
Afghanistan,64.023,61.640,59.367,57.170,0.000,0.000,0.000,0.000,0.000,0.000
Albania,11.803,10.807,9.943,9.267,0.000,0.000,0.000,0.000,0.000,0.000
Antigua and Barbuda,0.000,0.000,0.000,7.420,7.070,6.757,6.483,6.230,6.000,5.783
Argentina,0.000,0.000,0.000,11.283,10.733,10.203,9.683,9.177,8.680,8.227
Armenia,0.000,0.000,0.000,14.170,13.477,12.817,12.183,11.583,11.007,10.497
Australia,0.000,0.000,0.000,3.467,3.343,3.253,3.183,3.137,3.090,3.047
Austria,0.000,0.000,0.000,3.210,3.113,3.043,2.987,2.943,2.897,2.843


In [79]:
df5 = df.T.iloc[:7,:3]
df6 = df.T.iloc[6:,2:5]

display(df5)
display(df6)

Location,Afghanistan,Albania,Algeria
2010,64.023,11.803,23.540
2011,61.640,10.807,22.907
2012,59.367,9.943,22.450
2013,57.170,9.267,22.117
2014,55.080,8.790,21.850
2015,53.107,8.493,21.587
2016,51.267,8.363,21.257


Location,Algeria,Andorra,Angola
2016,21.257,3.187,55.510
2017,20.850,3.060,53.460
2018,20.407,2.933,51.757
2019,19.930,2.827,50.093


In [81]:
## Question 94

## df5과 df6 데이터를 하나의 데이터 프레임으로 merge함수를 이용하여 합쳐라. 
## Algeria컬럼을 key로 하고 두 데이터 모두 포함하는 데이터만 출력하라
Ans = pd.merge(df5,df6,on='Algeria',how='inner',left_index=True,right_index=True)
Ans

Location,Afghanistan,Albania,Algeria,Andorra,Angola
2016,51.267,8.363,21.257,3.187,55.51


In [82]:
## Question 95
## df5과 df6 데이터를 하나의 데이터 프레임으로 merge함수를 이용하여 합쳐라. Algeria컬럼을 key로 하고 합집합으로 합쳐라
Ans =pd.merge(df5,df6,on='Algeria',how='outer',left_index=True,right_index=True)
Ans

Location,Afghanistan,Albania,Algeria,Andorra,Angola
2010,64.023,11.803,23.540,NaN,NaN
2011,61.640,10.807,22.907,NaN,NaN
2012,59.367,9.943,22.450,NaN,NaN
2013,57.170,9.267,22.117,NaN,NaN
2014,55.080,8.790,21.850,NaN,NaN
2015,53.107,8.493,21.587,NaN,NaN
2016,51.267,8.363,21.257,3.187,55.510
2017,NaN,NaN,20.850,3.060,53.460
2018,NaN,NaN,20.407,2.933,51.757
2019,NaN,NaN,19.930,2.827,50.093
